In [26]:
# 1. Импортируем библиотеки
import pandas as pd 
import re

In [27]:
# 2. Считываем датафрейм.
path = ("_data.csv")
df = pd.read_csv(path)

In [28]:
# 3. Устнавливаем опцию отображения всех столбцов.
pd.set_option('display.max_columns', None)

In [29]:
# 4. Оставляем в датафрейме объявление только по Москве.
df = df[df["Адрес"].str.contains("Москва", case=False)]
len(df)

19737

In [30]:
# 5. Ищем дубликаты объявлений по описанию.
duplicate_rows = df[df.duplicated(subset='Описание')]
len(duplicate_rows)

281

In [31]:
# 6. Удаляем дубликаты объявлений по описанию.
df = df.drop_duplicates(subset='Описание')
len(df)

19456

In [32]:
# 7. Удаляем лишние столбцы, которые не повлияют или незначительно повлияют на результат.
df.drop(["Unnamed: 0", "Тип", "Телефоны", "Описание", "Площадь комнат, м2", "Дополнительно", "Название ЖК", "Серия дома", "Ссылка на объявление"], inplace=True, axis=1)

In [33]:
# 8. Меняем название колонок.
rename_map = {'Количество комнат': "number_of_rooms", 'Метро': "metro", 'Адрес': "address", 'Площадь, м2': "apartment_area_m2", 
              'Дом': "house", 'Парковка': "parking_lot", 'Можно с детьми/животными': 'Children/pets allowed','Высота потолков, м': 'Ceiling_height',
              'Лифт': 'Elevator','Мусоропровод': 'Garbage_chute', 'Цена': 'price', 'Ремонт': 'decor', 'Балкон': 'balcony', 'Окна': 'windows', 
              'Санузел': 'bathroom'}

df.rename(columns=rename_map, inplace=True)

In [34]:
# 9. Заполняем пропущенные значения
df["number_of_rooms"].fillna("Апартаменты/свободная планировка", inplace=True)
df["metro"].fillna("Нет_данных", inplace=True)
df["parking_lot"].fillna("Отсутствует", inplace=True)

df['ceiling_height'] = df['Ceiling_height'].fillna(df['Ceiling_height'].mean())
df['elevator'] = df['Elevator'].fillna('Нет данных')
df['garbage_chute'] = df['Garbage_chute'].fillna('Нет_данных')
df['children/pets allowed'] = df['Children/pets allowed'].fillna('Нет_данных')

df['decor'].fillna('Нет_данных', inplace=True)
df['balcony'].fillna('Отсутствует',inplace=True)
df['windows'].fillna('Нет_данных',inplace=True)
df['bathroom'].fillna('Нет_данных',inplace=True)

In [42]:
def filter_values(lst, st):
    for i in lst:
        if st in i:
            for z in i:
                if z in ['1','2','3','4','5','6','7','8','9']:
                    return int(z)

In [96]:
df['Children_yes'] = df['children/pets allowed'].apply(lambda x: 1 if 'Можно с детьми' in x else 0)
df['pets_yes'] = df['children/pets allowed'].apply(lambda x: 1 if 'Можно с животными' in x else 0)
df['Children_and_pets_yes'] = df['children/pets allowed'].apply(lambda x: 1 if x == 'Можно с детьми, Можно с животными' else 0)
df['Elevator_type'] = df['elevator'].apply(lambda x: len(x.split(',')))
df['Elevator_pass'] = df['elevator'].apply(lambda x: 0 if filter_values(x.split(','),'Пасс') == None else filter_values(x.split(','),'Пасс'))
df['Elevator_cargo'] = df['elevator'].apply(lambda x: 0 if filter_values(x.split(','),'Груз') == None else filter_values(x.split(','),'Груз'))
df['Garbage_chute_ yes'] = df['Garbage_chute'].apply(lambda x: 1 if x == 'Да' else 0)
df['bathroom_type']= df['bathroom'].apply(lambda x: len(x.split(',')))
df['bathroom_comb'] = df['bathroom'].apply(lambda x: 0 if filter_values(x.split(','),'Совмещенный') == None else filter_values(x.split(','),'Совмещенный'))
df['bathroom_separate'] = df['bathroom'].apply(lambda x: 0 if filter_values(x.split(','),'Раздельный') == None else filter_values(x.split(','),'Раздельный'))

df['bathroom_separate'] = df['bathroom'].apply(lambda x: 0 if filter_values(x.split(','),'Раздельный') == None else filter_values(x.split(','),'Раздельный'))
df['windows_street'] = df['windows'].apply(lambda x: 1 if 'На улицу' in x else 0)
df['windows_yard'] = df['windows'].apply(lambda x: 1 if 'Во двор' in x else 0)
df['windows_all'] = df['windows'].apply(lambda x: 1 if 'На улицу и двор' in x else 0)

df['balcony_type']= df['balcony'].apply(lambda x: 0 if x == 'Отсутствует' else len(x.split(',')))
df['balcony_bal']= df['balcony'].apply(lambda x: 0 if filter_values(x.split(','),'Балкон') == None else filter_values(x.split(','),'Балкон'))
df['balcony_loggia']= df['balcony'].apply(lambda x: 0 if filter_values(x.split(','),'Лоджия') == None else filter_values(x.split(','),'Лоджия'))

df['decor_designer']= df['decor'].apply(lambda x: 1 if x == 'Дизайнерский' else 0)
df['decor_evro']= df['decor'].apply(lambda x: 1 if x == 'Евроремонт' else 0)
df['decor_cosmetic']= df['decor'].apply(lambda x: 1 if x == 'Косметический' else 0)

df['parking_lot_under']= df['parking_lot'].apply(lambda x: 1 if x == 'подземная' else 0)
df['parking_lot_ground']= df['parking_lot'].apply(lambda x: 1 if x == 'наземная' else 0)
df['parking_lot_open']= df['parking_lot'].apply(lambda x: 1 if x == 'открытая' else 0)
df['parking_multi-level']= df['parking_lot'].apply(lambda x: 1 if x == 'многоуровневая' else 0)

In [110]:
print(df['house'].unique())

['5/16, Монолитный' '5/16, Монолитно-кирпичный' '5/16' ...
 '12/23, Панельный' '12/25, Блочный' '2/2, Блочный']


In [111]:
df.iloc[75:79]


,ID объявления,number_of_rooms,metro,address,apartment_area_m2,house,parking_lot,price,decor,balcony,windows,bathroom,Children/pets allowed,Ceiling_height,Elevator,Garbage_chute,ceiling_height,elevator,garbage_chute,children/pets allowed,Children_yes,pets_yes,Children_and_pets_yes,Elevator_type,Elevator_pass,Elevator_cargo,Garbage_chute_ yes,bathroom_type,bathroom_comb,bathroom_separate,windows_yard,windows_street,windows_all,balcony_type,balcony_bal,balcony_loggia,decor_designer,decor_evro,decor_cosmetic,parking_lot_under,parking_lot_ground,parking_lot_open,parking_multi-level
75,269165004,3,м. Смоленская (2 мин пешком),"Москва, 1-й Смоленский переулок, 21",92.3/10.8,"3/7, Монолитно-кирпичный",многоуровневая,"470000.0 руб./ За месяц, Залог - 1500000 руб.,...",Нет_данных,Отсутствует,Нет_данных,Раздельный (2),NaN,4.2,"Пасс (1), Груз (1)",Да,4.2,"Пасс (1), Груз (1)",Да,Нет_данных,0,0,0,2,1,1,1,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,1
76,274105576,"5, Оба варианта",м. Смоленская (5 мин пешком),"Москва, 1-й Смоленский переулок, 17",204.0/122.0/10.0,2/13,подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Евроремонт,Балкон (1),На улицу и двор,Совмещенный (4),"Можно с детьми, Можно с животными",3.2,Пасс (1),NaN,3.2,Пасс (1),Нет_данных,"Можно с детьми, Можно с животными",1,1,1,1,1,0,0,1,4,0,0,1,1,1,1,0,0,1,0,1,0,0,0
77,273785180,"4, Оба варианта",м. Арбатская (4 мин пешком),"Москва, улица Новый Арбат, 15",160.0/103.0/9.0,26/26,Отсутствует,"650000.0 руб./ За месяц, Залог - 650000 руб., ...",Евроремонт,Отсутствует,На улицу и двор,Совмещенный (3),"Можно с детьми, Можно с животными",3.2,Пасс (1),NaN,3.2,Пасс (1),Нет_данных,"Можно с детьми, Можно с животными",1,1,1,1,1,0,0,1,3,0,0,1,1,0,0,0,0,1,0,0,0,0,0
78,273785183,"4, Оба варианта",м. Арбатская (4 мин пешком),"Москва, улица Новый Арбат, 15",140.0/93.0/9.0,25/26,Отсутствует,"650000.0 руб./ За месяц, Залог - 650000 руб., ...",Евроремонт,Отсутствует,На улицу и двор,Совмещенный (3),"Можно с детьми, Можно с животными",3.2,Пасс (1),NaN,3.2,Пасс (1),Нет_данных,"Можно с детьми, Можно с животными",1,1,1,1,1,0,0,1,3,0,0,1,1,0,0,0,0,1,0,0,0,0,0


In [37]:
df.to_csv("data.csv")